# Image-Guided Generation

This notebook demonstrates how to use the content and layout of one image to influence the generation of a new one. The input image is called a "conditioning image". This is sometimes referred to as "ControlNet". This feature can be useful to gain more direct control over things like:
- image composition
- camera angle
- subject placement
- subject pose

Amazon image models support two types of conditioning image control modes:
- "CANNY_EDGE" - This mode retains the prominent outlines of objects in the conditioning image. This is most useful when you want the shape of the objects in the generated image to adhere closely to those in the conditioning image.
- "SEGMENTATION" - This mode allows the model more flexibility to change the shape of the objects it generates while still producing an identical image layout.

The code below uses the conditioning image on the left and the prompt *"3d animated film style, a woman with a crazy blond hair style, wearing a green sequin dress"*. Note that with CANNY_EDGE as the control mode, it includes edge-defined details like the glasses, the coverging vertical lines in the background, and maintains the exact collar shape. With SEGMENTATION mode, the model is free to be more creative.

<div style="display: flex; justify-content: space-between;">
    <div style="width: 31%;">
        <p align="center">
            <img src="../images/condition-image-1.png" width="100%" style="padding: 4px">
            <br>
            <em>Condition image</em>
        </p>
    </div>
    <div style="width: 31%;">
        <p align="center">
            <img src="../images/doc-images/image-conditioned-example-1.png" width="100%" style="padding: 4px">
            <br>
            <em>Output - CANNY_EDGE</em>
        </p>
    </div>
    <div style="width: 31%;">
        <p align="center">
            <img src="../images/doc-images/image-conditioned-example-2.png" width="100%" style="padding: 4px">
            <br>
            <em>Output - SEGMENTATION</em>
        </p>
    </div>
</div>


Experiment with changing the `controlStrength`, `controlMode`, and `text` prompt.

In [ ]:
from random import randint
from amazon_image_gen import BedrockImageGenerator
import file_utils
import logging
import base64
from datetime import datetime

logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')

conditioning_image_path = "../images/condition-image-1.png"

# Read image from file and encode it as base64 string.
with open(conditioning_image_path, "rb") as image_file:
    condition_image = base64.b64encode(image_file.read()).decode("utf8")


# Configure the inference parameters.
inference_params = {
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": "3d animated film style, a woman with a crazy blond hair style, wearing a green sequin dress",
        "conditionImage": condition_image,
        "controlMode": "SEGMENTATION", # "CANNY_EDGE" or "SEGMENTATION",
        "controlStrength": 0.3  # How closely to match the condition image
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,  # Number of variations to generate. 1 to 5.
        "quality": "standard",  # Allowed values are "standard" and "premium"
        "width": 1280,  # See README for supported output resolutions
        "height": 720,  # See README for supported output resolutions
        "cfgScale": 8.0,  # How closely the prompt will be followed
        "seed": randint(0, 858993459),  # Use a random seed
    },
}

# Define an output directory with a unique name.
generation_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_directory = f"output/{generation_id}"

# Create the generator.
generator = BedrockImageGenerator(
    output_directory=output_directory
)

# Generate the image(s).
response = generator.generate_images(inference_params)

if "images" in response:
    # Save and display each image
    images = file_utils.save_base64_images(response["images"], output_directory, "image")
    for image in images:
        display(image)